# Testing and Benchmarking

We load the best weights we've found, and load them in a model.

In [1]:
# Load best model weights
from keras.models import load_model
model = load_model('tmp/weights-39ep-16x106.hdf5')

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Unlike training we resize the images online here, it takes about 10 minutes.

In [5]:
import glob
from keras.preprocessing.image import img_to_array
from skimage.transform import resize
from skimage.io import imread

def preprocess_image(paths, i):
    sample_img = imread(paths[i])
    sample_img = sample_img.T[:,106:106*3,106:106*3]
    sample_img = resize(sample_img, (106, 106, 3), mode='reflect').T
    return sample_img

def fetch_test_images():
    while 1:
        for i in range(len(test_files)):
            image = preprocess_image(test_files,i)
            image = img_to_array(image)
            test = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            yield (test)
            
test_files = glob.glob('../data/images_test_rev1/*.jpg')

Generate predictions.

In [6]:
predictions = model.predict_generator(fetch_test_images(),
                       val_samples = len(test_files),
                        max_q_size = 32,)
predictions.shape # assert (79975, 37)

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `predict_generator` call to the Keras 2 API: `predict_generator(<generator..., steps=79975, max_queue_size=32)`
  This is separate from the ipykernel package so we can avoid doing imports until


(79975, 37)

Record all results into a csv file with the id and the predictions. The CSV file will be about 40MB big. We only load the all_zeros.csv file for its header.

In [9]:
from os import path

header = open('../data/all_zeros_benchmark.csv','r').readlines()[0]

with open('submission_1.csv','w') as outfile:
    outfile.write(header)
    for i in range(len(test_files)):
        fname = test_files[i]
        file_id = path.split(fname)[-1] \
                      .split('.')[0]
        pred = predictions[i]
        outline = file_id + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")